<a href="https://colab.research.google.com/github/Hemant9785/deep-learning-/blob/main/pruning_on_bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.datasets import mnist

# # Load the MNIST dataset
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# # Preprocess the data
# num_classes = 10
# input_shape = (28, 28)

# x_train = x_train.reshape(-1, *input_shape).astype("float32") / 255.0
# x_test = x_test.reshape(-1, *input_shape).astype("float32") / 255.0

# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# # Build the BiLSTM model
# model = keras.Sequential()
# InputLayer = tf.keras.Input(shape=input_shape)
# # Forward LSTM layer
# model.add(InputLayer)
# FW1 = layers.LSTM(28, return_sequences=True)
# model.add(FW1)
# # Backward LSTM layer
# BW1= layers.LSTM(28, return_sequences=True, go_backwards=True)
# model.add(BW1)
# BL1 = layers.Bidirectional(FW1,merge_mode='ave',backward_layer=BW1)(InputLayer)
# # Merge forward and backward layers
# model.add(layers.Bidirectional())
# # Output layer
# model.add(layers.Dense(num_classes, activation="softmax"))

# # Compile the model
# model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# # Train the model
# batch_size = 128
# epochs = 10
# model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# # Evaluate the model
# _, test_accuracy = model.evaluate(x_test, y_test)
# print("Test accuracy:", test_accuracy)


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
num_classes = 10
input_shape = (28, 28)

x_train = x_train.reshape(-1, *input_shape).astype("float32") / 255.0
x_test = x_test.reshape(-1, *input_shape).astype("float32") / 255.0

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Build the BiLSTM model
model = keras.Sequential()

# Forward LSTM layer
model.add(layers.LSTM(28, return_sequences=True, input_shape=input_shape))
# Backward LSTM layer
model.add(layers.LSTM(28, return_sequences=True, go_backwards=True))
# Merge forward and backward layers
model.add(layers.Bidirectional(layers.LSTM(28, return_sequences=True), merge_mode='ave'))
# Flatten the output
model.add(layers.Flatten())
# Output layer
model.add(layers.Dense(num_classes, activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
batch_size = 128
epochs = 3
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Evaluate the model
_, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy:", test_accuracy)



Epoch 1/3
422/422 [==============================] - 33s 55ms/step - loss: 0.6170 - accuracy: 0.7961 - val_loss: 0.1725 - val_accuracy: 0.9435
Epoch 2/3
422/422 [==============================] - 22s 52ms/step - loss: 0.1531 - accuracy: 0.9526 - val_loss: 0.1009 - val_accuracy: 0.9697
Epoch 3/3
313/313 [==============================] - 3s 9ms/step - loss: 0.0881 - accuracy: 0.9697
Test accuracy: 0.9696999788284302


In [3]:
model.save('model.h5')

In [4]:
# pip install tensorflow_model_optimization

In [5]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# from tensorflow_model_optimization.sparsity import keras as sparsity

# # Custom wrapper layer for Bidirectional layer
# class PrunableBidirectional(tf.keras.layers.Layer, sparsity.PrunableLayer):
#     def __init__(self, layer, **kwargs):
#         super(PrunableBidirectional, self).__init__(**kwargs)
#         self.layer = layer

#     def build(self, input_shape):
#         self.layer.build(input_shape)
#         self.built = True

#     def call(self, inputs):
#         return self.layer(inputs)

#     def compute_output_shape(self, input_shape):
#         return self.layer.compute_output_shape(input_shape)

#     def get_config(self):
#         return self.layer.get_config()

#     def get_prunable_weights(self):
#         return self.layer.forward_layer._trainable_weights  + self.layer.backward_layer._trainable_weights 

# # Load the saved model
# model = load_model('model.h5')

# # Wrap the Bidirectional layer with PrunableBidirectional
# prunable_model = tf.keras.Sequential()
# for layer in model.layers:
#     if isinstance(layer, tf.keras.layers.Bidirectional):
#         prunable_model.add(PrunableBidirectional(layer))
#     else:
#         prunable_model.add(layer)

# # Define the pruning parameters
# pruning_params = {
#     'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.90, begin_step=0, end_step=2000)
# }

# # Apply pruning to the model
# pruned_model = sparsity.prune_low_magnitude(prunable_model, **pruning_params)

# # Compile the pruned model
# pruned_model.compile(
#     loss="categorical_crossentropy",
#     optimizer="adam",
#     metrics=["accuracy"]
# )

# # Train the pruned model (optional)
# pruned_model.fit(x_train, y_train, batch_size=128, epochs=1, validation_split=0.1)

# # Save the pruned model
# pruned_model.save('pruned_model.h5')


In [7]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow_model_optimization.sparsity import keras as sparsity

# Custom wrapper layer for Bidirectional layer
class PrunableBidirectional(tf.keras.layers.Layer, sparsity.PrunableLayer):
    def __init__(self, layer, **kwargs):
        super(PrunableBidirectional, self).__init__(**kwargs)
        self.layer = layer

    def build(self, input_shape):
        self.layer.build(input_shape)
        self.built = True

    def call(self, inputs):
        return self.layer(inputs)

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)

    def get_config(self):
        return self.layer.get_config()

    def get_prunable_weights(self):
        return self.layer.forward_layer._trainable_weights + self.layer.backward_layer._trainable_weights

# Load the saved model
model = load_model('model.h5')

# Wrap the Bidirectional layer with PrunableBidirectional
prunable_model = tf.keras.Sequential()
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Bidirectional):
        prunable_model.add(PrunableBidirectional(layer))
    else:
        prunable_model.add(layer)

# Define the pruning parameters
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.90, begin_step=0, end_step=2000)
}

# Apply pruning to the model
pruned_model = sparsity.prune_low_magnitude(prunable_model, **pruning_params)

# Define the pruning callbacks
callbacks = [
    sparsity.UpdatePruningStep(),
]

# Compile the pruned model
pruned_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Train the pruned model
pruned_model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=2,
    validation_split=0.1,
    callbacks=callbacks
)

# Save the pruned model
# pruned_model.save('pruned_model2.h5')


Epoch 1/2
422/422 [==============================] - 31s 55ms/step - loss: 0.1228 - accuracy: 0.9612 - val_loss: 0.1254 - val_accuracy: 0.9625
Epoch 2/2
422/422 [==============================] - 22s 51ms/step - loss: 0.1443 - accuracy: 0.9553 - val_loss: 0.1247 - val_accuracy: 0.9645


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 28)            6384      
                                                                 
 lstm_1 (LSTM)               (None, 28, 28)            6384      
                                                                 
 bidirectional (Bidirectiona  (None, 28, 28)           12768     
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 33,386
Trainable params: 33,386
Non-trainable params: 0
____________________________________________________

In [10]:
pruned_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_lstm (P  (None, 28, 28)           12659     
 runeLowMagnitude)                                               
                                                                 
 prune_low_magnitude_lstm_1   (None, 28, 28)           12659     
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_prunabl  (None, 28, 28)           12769     
 e_bidirectional_1 (PruneLow                                     
 Magnitude)                                                      
                                                                 
 prune_low_magnitude_flatten  (None, 784)              1         
  (PruneLowMagnitude)                                            
                                                      

In [18]:
import tensorflow_model_optimization as tfmot
# tfmot.sparsity.keras.prune_low_magnitude.summary_weights(pruned_model)
import numpy as np

# Iterate over the layers of the pruned model
for layer in pruned_model.layers:
    if isinstance(layer, PrunableBidirectional):
        forward_weights = layer.layer.forward_layer.get_weights()
        backward_weights = layer.layer.backward_layer.get_weights()
        print(forward_weights)
        print(1)
        forward_sparsity = 1 - np.count_nonzero(forward_weights[0]) / forward_weights[0].size
        backward_sparsity = 1 - np.count_nonzero(backward_weights[0]) / backward_weights[0].size
        
        print("Forward Layer Sparsity: {:.2f}%".format(forward_sparsity * 100))
        print("Backward Layer Sparsity: {:.2f}%".format(backward_sparsity * 100))
        print()
    else:
        weights = layer.get_weights()
        if weights:
            # print(weights)
            sparsity = 1 - np.count_nonzero(weights[0]) / weights[0].size
            print("Layer Sparsity: {:.2f}%".format(sparsity * 100))
            print()
        else:
            print("Layer has no trainable weights.")
            print()


[array([[ 0.        ,  0.        , -0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.24565847,  0.        ],
       [ 0.        , -0.        ,  0.        , ...,  0.        ,
         0.        ,  0.27681792],
       ...,
       [-0.28916886, -0.        , -0.        , ..., -0.        ,
        -0.27141735,  0.        ],
       [-0.        , -0.        ,  0.33018345, ...,  0.        ,
        -0.        ,  0.28975338],
       [-0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.        ,  0.        ]], dtype=float32), array([[ 0.        ,  0.34553406, -0.        , ...,  0.        ,
        -0.        ,  0.        ],
       [ 0.27240983, -0.        , -0.        , ...,  0.        ,
        -0.28138286, -0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.27839214,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.27253476,  0.        , ..., -0.   